# Train autoencoder
This notebook is made to train an autoencoder or a variational autoencoder on the mnist data set. It can be run locally or on golab. Checks have been implemented for colab use. 

## Check if in Colab

In [26]:
import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

print("Is in Colab: ", IN_COLAB)
if IN_COLAB:
    os.system('git clone https://github.com/AllaVinner/JL-ML.git')
    os.system('pip install -e JL-ML')
    import site
    site.main()

Is in Colab:  False


## Setup

In [19]:
#Test to load

import tensorflow as tf
from tensorflow import keras
import numpy as np

from jlauto.models.variational_autoencoder import VariationalAutoencoder
from jlauto.models.autoencoder import Autoencoder
from jlauto.models.load_premade import load_premade_model
from jlauto.models.continuous_bernoulli_loss import continuous_bernoulli_loss

In [20]:
#Preprocess mnist data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
#num_samples = 100
train_digits = np.expand_dims(x_train, -1).astype("float32") / 255
input_shape = train_digits.shape[1:]

In [21]:

##############################################################################
model_type = 'autoencoder'
model_name = 'mnist_cnn_shallow'
optimizer  = 'adam'
epochs     = 1
latent_dims = [2,4,8,16]
names       = ['ae_latent_dim_'+str(lat_dim) for lat_dim in latent_dims]
save_path  = '../saved-models/'


for i in range(len(names)):
  latent_dim = latent_dims[i]
  name = names[i]  
  # Create and train model
  model = load_premade_model(model_type = model_type,
                            model_name = model_name,
                            input_shape = input_shape,
                            latent_dim = latent_dim)

  model.compile(optimizer = optimizer,loss = 'binary_crossentropy')

  model.fit(train_digits,train_digits,
            epochs = epochs,
            batch_size = 512)

  model.save(save_path + name)


In [27]:
# Set default configure 
config = {
    'name': 'CHANGE',
    'model_type': 'autoencoder',
    'model_name': 'mnist_cnn_shallow',
    'latent_dim': 'CHANGE',
    'optimizer': 'adam',
    'loss': 'binary_crossentropy', 
    'batch_size': 512,
    'epochs': 1,
}

In [28]:
# Changing parameters
changing_config = {}
changing_config['latent_dim'] = [3,4]
changing_config['name']   = ['ae_latent_dim_'+str(lat_dim) for lat_dim in changing_config['latent_dim']]


## Train

In [35]:
for i in range(len(changing_config['name'])):
  # update config
  for key, values in changing_config.items():
    config[key] = values[i]
  
  # Save config
  model_path = os.path.join(save_path,config['name'])
  os.system(f'mkdir {model_path}')
  with open(os.path.join(save_path,config['name'],'config.yaml'), 'w') as yaml_file:
    yaml.dump(config, yaml_file)

  # Create and train model
  model = load_premade_model(model_type = config['model_type'],
                            model_name = config['model_name'],
                            input_shape = input_shape,
                            latent_dim = config['latent_dim'])

  model.compile(optimizer = config['optimizer'],loss = config['loss'])

  model.fit(train_digits,train_digits,
            epochs = config['epochs'],
            batch_size = config['batch_size'])

  model.save(os.path.join(save_path, config['name']))


(28, 28, 1)
118/118 [==============================] - 43s 341ms/step - loss: 0.3422
INFO:tensorflow:Assets written to: ..\saved-models\ae_latent_dim_3\assets
(28, 28, 1)
118/118 [==============================] - 41s 326ms/step - loss: 0.3464
INFO:tensorflow:Assets written to: ..\saved-models\ae_latent_dim_4\assets


## Zip if in colab

In [ ]:
if IN_COLAB:
    os.system('zip -r ./models.zip {save_path}')

# Investigate model

In [ ]:
# Fill
digit_df_list = []
dimension_df_list = []
#latent_df_list = []


for name, model in model_df.iterrows():
    print(name)
    if model['ae_type'] == "ae":
        code = model['model'].encoder(test_digits).numpy()
        code_std = np.empty(shape = code.shape)*np.nan
        
    elif model['ae_type'] == "vae":
        digit_distribution = model['model'].encoder(test_digits).numpy()
        code = digit_distribution[:,0,:]
        code_std = np.sqrt(np.exp(digit_distribution[:,1,:]))
    
    # Caclualte digit_df data
    digit_distance = np.linalg.norm(code, axis = 1)
    digit_radius = np.power(np.prod(code_std, axis = 1),1/latent_dim)
    digit_loss = tf.reduce_mean(tf.reduce_mean(tf.keras.losses.binary_crossentropy(test_digits, model['model'](test_digits)),axis = -1),axis = -1).numpy()
    digit_tsne = TSNE(n_components = 2).fit_transform(code)
    
    _digit_df = pd.DataFrame(data = {
        'model_name' : name,
        'label' : test_labels,
        'loss' : digit_loss,
        'distance' : digit_distance,
        'radius' : digit_radius,
        'tsne_0' : digit_tsne[:,0],
        'tsne_1' : digit_tsne[:,1],
    })
    digit_df_list.append(_digit_df)   
    
    
    # Calculate dimension_df data
    value_spread = np.nanstd(code, axis = 0)
    spread_mean = np.nanmean(code_std, axis = 0)
    dim_index = np.arange(latent_dim)
    
    _dimension_df = pd.DataFrame(data = {
        'model_name' : name,
        'dim_index' : dim_index,
        'value_spread' : value_spread,
        'spread_mean' : spread_mean,
    })
    
    dimension_df_list.append(_dimension_df)
    """
    # Calculate latent_df data
    latent_labels = test_labels.repeat(repeats = latent_dim)
    latent_digit_index = np.arange(num_test).repeat(repeats = latent_dim)
    latent_dim_index = np.tile(np.arange(latent_dim), num_test)
    latent_value = code.reshape(-1)
    latent_spread = code_std.reshape(-1)
    
    _latent_df = pd.DataFrame(data = {
        'model_name' : name,
        'label' : latent_labels,
        'digit_index' : latent_digit_index,
        'dim_index' : latent_dim_index,
        'value' : latent_value,
        'spread' : latent_spread
    })
    
    latent_df_list.append(_latent_df)
    """
    
        
digit_df = pd.concat(digit_df_list)
dimension_df = pd.concat(dimension_df_list)
#latent_df = pd.concat(latent_df_list)